# Bi-Directional LSTM Implementation
Source:https://www.kaggle.com/c/fake-news/data

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df.shape

(20800, 5)

In [7]:
df=df.dropna()

In [8]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [9]:
#Get independent features
X=df.drop('label',axis=1)

In [10]:
y=df['label']

In [11]:
y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [12]:
import tensorflow as tf

In [13]:
print(tf.__version__)

2.15.0


In [16]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [17]:
## defining a vocabulary size
voc_size=3000


In [32]:
messages=X.copy()

In [33]:
import nltk
import re
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [34]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:
messages.reset_index(inplace=True)

In [37]:
## Applying preprocessing
corpus=[]
for i in range(0,len(messages)):
  review=re.sub('[^a-zA-Z]',' ',messages['title'][i])
  review=review.lower()
  review=review.split()
  review=[ps.stem(word) for word in review if not word in (stopwords.words('english'))]
  review=' '.join(review)
  corpus.append(review)

In [39]:
corpus[:10]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag']

In [40]:
#doing one_hot representation

ohe=[one_hot(words,voc_size)for words in corpus]


In [41]:
print(ohe[:5])

[[323, 2109, 464, 1724, 264, 1364, 36, 1116, 2394, 1433], [2696, 10, 1914, 2308, 1089, 2012, 2420], [2575, 885, 884, 2510], [937, 2043, 2248, 2228, 1084, 1892], [1984, 1089, 1601, 2796, 2812, 2986, 1089, 2880, 1125, 1374]]


In [42]:
#prepadding(Embedding Representation)
sent_length=30
embedded_docs=pad_sequences(ohe,padding='pre',maxlen=sent_length)
print(embedded_docs[:5])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0  323 2109  464 1724  264 1364   36 1116
  2394 1433]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0 2696   10 1914 2308 1089
  2012 2420]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0 2575  885
   884 2510]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0  937 2043 2248 2228
  1084 1892]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0 1984 1089 1601 2796 2812 2986 1089 2880
  1125 1374]]


In [52]:
#Build a model
embedded_vec_features=30 #after padding the indexes would be converted to vectors
model=Sequential()
model.add(Embedding(voc_size,embedded_vec_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 30, 30)            90000     
                                                                 
 bidirectional_2 (Bidirecti  (None, 200)               104800    
 onal)                                                           
                                                                 
 dense_4 (Dense)             (None, 1)                 201       
                                                                 
Total params: 195001 (761.72 KB)
Trainable params: 195001 (761.72 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [48]:
import numpy as np

In [49]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [47]:
from sklearn.model_selection import train_test_split

In [50]:
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.20,random_state=0)

In [53]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
229/229 [==============================] - 38s 142ms/step - loss: 0.3239 - accuracy: 0.8399 - val_loss: 0.2248 - val_accuracy: 0.9130
Epoch 2/10
229/229 [==============================] - 40s 175ms/step - loss: 0.1693 - accuracy: 0.9312 - val_loss: 0.1864 - val_accuracy: 0.9240
Epoch 3/10
229/229 [==============================] - 36s 156ms/step - loss: 0.1238 - accuracy: 0.9531 - val_loss: 0.1831 - val_accuracy: 0.9253
Epoch 4/10
229/229 [==============================] - 34s 150ms/step - loss: 0.0895 - accuracy: 0.9679 - val_loss: 0.2133 - val_accuracy: 0.9218
Epoch 5/10
229/229 [==============================] - 37s 163ms/step - loss: 0.0648 - accuracy: 0.9799 - val_loss: 0.2337 - val_accuracy: 0.9270
Epoch 6/10
229/229 [==============================] - 33s 144ms/step - loss: 0.0454 - accuracy: 0.9878 - val_loss: 0.2495 - val_accuracy: 0.9232
Epoch 7/10
229/229 [==============================] - 23s 99ms/step - loss: 0.0323 - accuracy: 0.9910 - val_loss: 0.3214 - val_acc

In [54]:
y_pred=model.predict(X_test)

115/115 [==============================] - 3s 12ms/step


In [57]:
y_pred=np.where(y_pred>=0.5,1,0)

In [58]:
from sklearn.metrics import accuracy_score,classification_report
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.9149576155318567
              precision    recall  f1-score   support

           0       0.92      0.93      0.92      2040
           1       0.91      0.90      0.90      1617

    accuracy                           0.91      3657
   macro avg       0.91      0.91      0.91      3657
weighted avg       0.91      0.91      0.91      3657

